# **Boosting Wav2Vec2 with n-grams in 🤗 Transformers**

**Wav2Vec2** is a popular pre-trained model for speech recognition. Released in [September 2020](https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/) by Meta AI Research, the novel architecture catalyzed progress in self-supervised pretraining for speech recognition, *e.g.* [*G. Ng et al.*, 2021](https://arxiv.org/pdf/2104.03416.pdf), [*Chen et al*, 2021](https://arxiv.org/abs/2110.13900), [*Hsu et al.*, 2021](https://arxiv.org/abs/2106.07447) and [*Babu et al.*, 2021](https://arxiv.org/abs/2111.09296). On the Hugging Face Hub, Wav2Vec2's most popular pre-trained checkpoint currently amounts to over [**250,000** monthly downloads](https://huggingface.co/facebook/wav2vec2-base-960h).

Using Connectionist Temporal Classification (CTC), pre-trained Wav2Vec2-like checkpoints are extremely easy to fine-tune on downstream speech recognition tasks.
In a nutshell, fine-tuning pre-trained Wav2Vec2 checkpoints works as follows: 

A single randomly initialized linear layer is stacked on top of the pre-trained checkpoint and trained to classify raw audio input to a sequence of letters. It does so by:

1.  extracting audio representations from the raw audio (using CNN layers),
2. processing the sequence of audio representations with a stack of transformer layers, and,
3. classifying the processed audio representations into a sequence of output letters.

Previously audio classification models required an additional language model (LM) and a dictionary to transform the sequence of classified audio frames to a coherent transcription.
Wav2Vec2's architecture is based on transformer layers, thus giving each processed audio representation context 
from all other audio representations. In addition, 
Wav2Vec2 leverages the [CTC algorithm](https://distill.pub/2017/ctc/) for fine-tuning, which solves the problem of alignment between a varying "input audio length"-to-"output text length" ratio.

Having contextualized audio classifications and no alignment problems, Wav2Vec2 does not require 
an external language model or dictionary to yield acceptable audio transcriptions.

As can be seen in Appendix C of the [official paper](https://arxiv.org/abs/2006.11477), Wav2Vec2 gives impressive downstream performances on [LibriSpeech]((https://huggingface.co/datasets/librispeech_asr)) without using a language model at all. However, from the appendix, it also becomes clear that using Wav2Vec2 in combination with a language model can yield a significant improvement, especially when the model was trained on only 10 minutes of transcribed audio.

Until recently, the 🤗 Transformers library did not offer a simple user interface to decode audio files with a fine-tuned Wav2Vec2 **and** a language model. This has thankfully changed. 🤗 Transformers now offers an easy-to-use integration with *Kensho Technologies'* [pyctcdecode library](https://github.com/kensho-technologies/pyctcdecode). This blog post is a step-by-step **technical** guide to explain how one can create an **n-gram** language model and combine it with an existing fine-tuned Wav2Vec2 checkpoint using 🤗 Datasets and 🤗 Transformers.

We start by:

1. How does decoding audio with an LM differ from decoding audio without an LM?
2. How to get suitable data for a language model?
3. How to build an *n-gram* with KenLM?
4. How to combine the *n-gram* with a fine-tuned Wav2Vec2 checkpoint?

For a deep dive into how Wav2Vec2 functions - which is not necessary for this blog post - the reader is advised to consult the following material:

- [wav2vec 2.0: A Framework for Self-Supervised Learning of Speech Representations](https://arxiv.org/abs/2006.11477)
- [Fine-Tune Wav2Vec2 for English ASR with 🤗 Transformers](https://huggingface.co/blog/fine-tune-wav2vec2-english)
- [An Illustrated Tour of Wav2vec 2.0](https://jonathanbgn.com/2021/09/30/illustrated-wav2vec-2.html)

## **1. Decoding audio data with Wav2Vec2 and a language model**

As shown in 🤗 Transformers [exemple docs of Wav2Vec2](https://huggingface.co/docs/transformers/master/en/model_doc/wav2vec2#transformers.Wav2Vec2ForCTC), audio can be transcribed as follows.

We install `datasets` and `transformers` as well as `pyctcdecode` and `kenLM`'s Python bindings to be able to run the language model integration.



In [2]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install https://github.com/kpu/kenlm/archive/master.zip 
!pip install pyctcdecode==0.3.0
!pip install datasets #==2.0.0
!pip install transformers==4.20.0
!pip install urduhack
!pip install huggingface_hub
!pip install audioread
!pip install librosa

In [3]:
!pip install torch==1.11.0 torchaudio==0.11.0 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


In [5]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


Let's load a small excerpt of the [Librispeech dataset](https://huggingface.co/datasets/librispeech_asr) to demonstrate Wav2Vec2's speech transcription capabilities.

In [6]:
!git config --global credential.helper store
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [60]:
from datasets import load_dataset

dataset = load_dataset("mozilla-foundation/common_voice_8_0", "ur", split="train+validation", use_auth_token=True)
dataset

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 810
})

In [61]:
from datasets import Audio
dataset = dataset.cast_column("audio", Audio(sampling_rate=16_000))

In [62]:
dataset[0]

{'client_id': '9b429f257f586530550730449045b59f09cf6f278e7e170f4b93de7114c708d938d51214275a8be24c9f6459e65e792b3a6be8c4fcb6332f2c1a768ed3c5af40',
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/a06a0d310e758fb842573373d89f6cf9b011859e285bd5ea4bdb4f4518b9a760/cv-corpus-8.0-2022-01-19/ur/clips/common_voice_ur_26630004.mp3',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/a06a0d310e758fb842573373d89f6cf9b011859e285bd5ea4bdb4f4518b9a760/cv-corpus-8.0-2022-01-19/ur/clips/common_voice_ur_26630004.mp3',
  'array': array([ 0.000000e+00,  0.000000e+00,  0.000000e+00, ...,  9.434595e-06,
         -3.136347e-04, -3.876609e-04], dtype=float32),
  'sampling_rate': 16000},
 'sentence': 'پر اپنی محنت سے حاصل کیا ہے',
 'up_votes': 2,
 'down_votes': 0,
 'age': 'thirties',
 'gender': 'female',
 'accent': '',
 'locale': 'ur',
 'segment': ''}

In [63]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(dataset)-1)

print(dataset[rand_int]["sentence"])
ipd.Audio(data=dataset[rand_int]["audio"]["array"], autoplay=True, rate=16000)

ہر چیز میں ملاوٹ ہے۔


We can pick one of the 73 audio samples and listen to it.

In [64]:
list=[]
for i in range(0,len(dataset)-1):
  x= dataset[i]
  y=(x["sentence"])
  list.append(x)
  

In [65]:
import numpy as np
Array=np.array(list)
Array[1]

{'client_id': '9b429f257f586530550730449045b59f09cf6f278e7e170f4b93de7114c708d938d51214275a8be24c9f6459e65e792b3a6be8c4fcb6332f2c1a768ed3c5af40',
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/a06a0d310e758fb842573373d89f6cf9b011859e285bd5ea4bdb4f4518b9a760/cv-corpus-8.0-2022-01-19/ur/clips/common_voice_ur_26630005.mp3',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/a06a0d310e758fb842573373d89f6cf9b011859e285bd5ea4bdb4f4518b9a760/cv-corpus-8.0-2022-01-19/ur/clips/common_voice_ur_26630005.mp3',
  'array': array([0.        , 0.        , 0.        , ..., 0.00018168, 0.00053833,
         0.00115278], dtype=float32),
  'sampling_rate': 16000},
 'sentence': 'جبکہ مصباح الحق اور یونس خان جیسے سینئرز صرف اس لیے اب تک کھیل رہے ہیں',
 'up_votes': 2,
 'down_votes': 0,
 'age': 'thirties',
 'gender': 'female',
 'accent': '',
 'locale': 'ur',
 'segment': ''}

In [249]:
# import IPython.display as ipd

# audio_sample = dataset[6]
# print(audio_sample["sentence"])
# ipd.Audio(data=audio_sample["audio"]["array"], autoplay=True, rate=audio_sample["audio"]["sampling_rate"])

Having chosen a data sample, we now load the fine-tuned model and processor.

In [66]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained("BakhtUllah123/xls-r_cv_ur")
model = Wav2Vec2ForCTC.from_pretrained("BakhtUllah123/xls-r_cv_ur")

Next, we process the data

In [67]:
inputs=[]
for i in Array:
  input=processor(i["audio"]["array"], sampling_rate=16_000, return_tensors="pt")
  inputs.append(input)

In [252]:
# inputs = processor(audio_sample["audio"]["array"], sampling_rate=16_000, return_tensors="pt")
# inputs

forward it to the model

In [68]:
import torch
logits=[]
for i in range(len(inputs)):
  with torch.no_grad():
    logit = model(**inputs[0]).logits
  logits.append(logit)
  inputs.remove(inputs[0])

In [ ]:
# import torch

# with torch.no_grad():
#   logits = model(**inputs).logits

and decode it

In [69]:
transcriptionsWithoutLM=[]
used_logits = logits
for i in range(len(logits)):
  pred_ids=torch.argmax(logits[0],dim=-1)
  transcription=processor.batch_decode(pred_ids)
  transcriptionsWithoutLM.append(transcription)
  logits.remove(logits[0])


In [70]:
WithoutLM=[]
for i in range(len(transcriptionsWithoutLM)):
  WithoutLM.append(transcriptionsWithoutLM[i][0])

In [256]:
# predicted_ids = torch.argmax(logits, dim=-1)
# transcription = processor.batch_decode(predicted_ids)

# transcription[0].lower()

Comparing the transcription to the target transcription above, we can see that some words *sound* correct, but are not *spelled* correctly, *e.g.*:

- *christmaus* vs. *christmas*
- *rose* vs. *roast*
- *simalyis* vs. *similes*

Let's see whether combining Wav2Vec2 with an ***n-gram*** lnguage model can help here.

For demonstration purposes, we have prepared a new model repository [patrickvonplaten/wav2vec2-base-100h-with-lm](https://huggingface.co/patrickvonplaten/wav2vec2-base-100h-with-lm) which contains the same Wav2Vec2 checkpoint but has an additional **4-gram** language model for English.

Instead of using `Wav2Vec2Processor`, this time we use `Wav2Vec2ProcessorWithLM` to load the **4-gram** model in addition to the feature extractor and tokenizer.

In [71]:
from transformers import Wav2Vec2ProcessorWithLM

processor = Wav2Vec2ProcessorWithLM.from_pretrained("BakhtUllah123/xls-r_cv_ur")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:92: FutureWarning: Deprecated argument(s) used in 'snapshot_download': allow_regex. Will not be supported from version '0.12'.

Please use `allow_patterns` and `ignore_patterns` instead.
  warnings.warn(message, FutureWarning)


In constrast to decoding the audio without language model, the processor now directly receives the model's output `logits` instead of the `argmax(logits)` (called `predicted_ids`) above. The reason is that when decoding with a language model, at each time step, the processor takes the probabilities of all possible output characters into account. Let's take a look at the dimension of the `logits` output.

In [72]:
inputs=[]
for i in Array:
  input=processor(i["audio"]["array"], sampling_rate=16_000, return_tensors="pt")
  inputs.append(input)


In [73]:
import torch
logits=[]
for i in range(len(inputs)):
  with torch.no_grad():
    logit = model(**inputs[0]).logits
  logits.append(logit)
  inputs.remove(inputs[0])

In [ ]:
#logits.shape

We can see that the `logits` correspond to a sequence of 624 vectors each having 32 entries. Each of the 32 entries thereby stands for the logit probability of one of the 32 possible output characters of the model:

In [74]:
" ".join(sorted(processor.tokenizer.get_vocab()))

'</s> <s> [PAD] [UNK] | ء ا ب ت ث ج ح خ د ذ ر ز س ش ص ض ط ظ ع غ ف ق ل م ن و ى ي ً ٓ ٹ پ چ ڈ ڑ ژ ک گ ں ھ ہ ی ے'

Intuitively, one can understand the decoding process of `Wav2Vec2ProcessorWithLM` as applying beam search through a matrix of size 624 $\times$ 32 probabilities while leveraging the probabilities of the next letters as given by the *n-gram* language model.

OK, let's run the decoding step again. `pyctcdecode` language model decoder does not automatically convert `torch` tensors to `numpy` so we'll have to convert them ourselves before.

In [75]:
transcriptionWithLM=[]
for i in range(len(logits)):
  transcription = processor.batch_decode(logits[0].numpy()).text
  transcriptionWithLM.append(transcription)
  logits.remove(logits[0])


In [76]:
transcriptionWithLM[0]

['پر اپنی محنت سے حاصل کیا ہے']

In [77]:
WithLM=[]
for i in range(len(transcriptionWithLM)):
  WithLM.append(transcriptionWithLM[i][0])

In [261]:
# transcription = processor.batch_decode(logits.numpy()).text
# transcription[0].lower()

In [78]:
from jiwer import wer, cer

word_Error_Rate = wer(WithoutLM , WithLM) # computing WER by comparing refrence and prediction sentences
Character_Error_Rate = cer(WithoutLM , WithLM)   # computing CER by comparing refrence and prediction sentences

print("Word error rate is = {}".format(word_Error_Rate))
print("Character error rate is = {}".format(Character_Error_Rate))

Word error rate is = 0.03312863606981254
Character error rate is = 0.006522774744402507


Cool! Recalling the words `facebook/wav2vec2-base-100h` without a language model transcribed incorrectly previously, *e.g.*,

> - *christmaus* vs. *christmas*
- *rose* vs. *roast*
- *simalyis* vs. *similes*

we can take another look at the transcription of `facebook/wav2vec2-base-100h` **with** a 4-gram language model. 2 out of 3 errors are corrected; *christmas* and *similes* have been correctly transcribed.

Interestingly, the incorrect transcription of *rose* persists. However, this should not surprise us very much. Decoding audio without a language model is much more prone to yield spelling mistakes, such as *christmaus* or *similes* (those words don't exist in the English language as far as I know). This is because the speech recognition system almost solely bases its prediction on the acoustic input it was given and not really on the language modeling context of previous and successive predicted letters ${}^1$. 
If on the other hand, we add a language model, we can be fairly sure that the speech recognition system will heavily reduce spelling errors since a well-trained *n-gram* model will surely not predict a word that has spelling errors. But the word *rose* is a valid English word and therefore the 4-gram will predict this word with a probability that is not insignificant. 

The language model on its own most likely does favor the correct word *roast* since the word sequence *roast beef* is much more common in English than *rose beef*. Because the final transcription is derived from a weighted combination of `facebook/wav2vec2-base-100h` output probabilities and those of the *n-gram* language model, it is quite common to see incorrectly transcribed words such as *rose*.

For more information on how you can tweak different parameters when decoding with `Wav2Vec2ProcessorWithLM`, please take a look at the official documentation [here](https://huggingface.co/docs/transformers/master/en/model_doc/wav2vec2#transformers.Wav2Vec2ProcessorWithLM.batch_decode).

---
${}^1$ Some research shows that a model such as `facebook/wav2vec2-base-100h` - when sufficiently large and trained on enough data - can learn language modeling dependencies between intermediate audio representations similar to a language model.


Great, now that you have seen the advantages adding an *n-gram* language model can bring, let's dive into how to create an *n-gram* and `Wav2Vec2ProcessorWithLM` from scratch.

## **2. Getting data for your language model**

A language model that is useful for a speech recognition system should support the acoustic model, *e.g.* Wav2Vec2, in predicting the next word (or token, letter) and therefore model the following distribution:

$\mathbf{P}(w_n | \mathbf{w}_0^{t-1})$ with $w_n$ being the next word and $\mathbf{w}_0^{t-1}$ being the sequence of all previous words since the beginning of the utterance. Simply said, the language model should be good at predicting the next word given all previously transcribed words regardless of the audio input given to the speech recognition system.

As always a language model is only as good as the data it is trained on. In the case of speech recognition, we should therefore ask ourselves for what kind of data, the speech recognition will be used for: *conversations*, *audiobooks*, *movies*, *speeches*, *, etc*, ...?

The language model should be good at modeling language that corresponds to the 
target transcriptions of the speech recognition system. 
For demonstration purposes, we assume here that we have fine-tuned a pre-trained [`facebook/wav2vec2-xls-r-300m`](https://huggingface.co/facebook/wav2vec2-xls-r-300m) on [Common Voice 7](https://huggingface.co/datasets/mozilla-foundation/common_voice_7_0) in Swedish. The fine-tuned checkpoint can 
be found [here](https://huggingface.co/hf-test/xls-r-300m-sv).
Common Voice 7 is a relatively crowd-sourced read-out audio dataset and we will evaluate the model on its test data.

Let's now look for suitable text data on the Hugging Face Hub. We search all datasets for those [that contain Swedish data](https://huggingface.co/datasets?languages=languages:sv&sort=downloads). 
Browsing a bit through the datasets, we are looking for a dataset that is similar to Common Voice's read-out audio data. The obvious choices of [oscar](https://huggingface.co/datasets/oscar) and [mc4](https://huggingface.co/datasets/mc4) might not be the most suitable here because they:

- are generated from crawling the web, which might not be very clean and correspond well to spoken language
- require a lot of pre-processing
- are very large which is not ideal for demonstration purposes here 😉

A dataset that seems sensible here and which is relatively clean and easy to pre-process is [europarl_bilingual](https://huggingface.co/datasets/europarl_bilingual) as it's a dataset that is based on discussions and talks of the European parliament. It should therefore be relatively clean and correspond well to read-out audio data. The dataset is originally designed for machine translation and can therefore only be accessed in translation pairs. We will only extract the text of the target language, Swedish (`sv`), from the *English-to-Swedish* translations.

In [28]:
from datasets import load_dataset

dataset = load_dataset("mozilla-foundation/common_voice_8_0", "ur", split="train+validation", use_auth_token=True)
dataset

In [29]:
dataset = dataset.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes",'path', 'audio'])

In [ ]:
# target_lang="sv"  # change to your target lang

In [30]:
chars_to_ignore_regex = '[\۔\!\,\؟\:\-\“\%\‘\’\”\،\"\…\ﷺ\`\%\‘\’\”\'َ\'ُ\'ِ\'ّ\'ٔ\'ٰ|]'  # change to the ignored characters of your fine-tuned model

In [31]:
import re

def extract_text(batch):
  text = batch["sentence"]
  batch["text"] = re.sub(chars_to_ignore_regex, "", text.lower())
  return batch

,sentence
0,پچھلے ہفتے ڈرائیور بیمار تھا۔
1,وہ اپنی عمر سے چھوٹا لگتا ہے
2,پڑھے لکھے تو ہیں۔
3,زندگی سے محبت کرتا ہوں
4,منٹوں میں آرام مگر آزمائش شرط ہے ۔
5,آپ یہاں سے یہ کتابچہ مفت حاصل کرسکتے ہیں
6,ایف بی ار میں چھٹیوں پر پابندی عائد
7,چوک کراس کیجئے
8,ان چھ کے ہی آٹھ آملیٹ تیار کر لیں گے
9,ہاؤس افیکٹ کو نبات خانہ کا اثر


In [32]:
dataset = dataset.map(extract_text)
dataset

In [33]:
dataset[4]

In [34]:
from huggingface_hub import notebook_login

notebook_login()

,sentence
0,کا چیف جسٹس صاحب نے کہا ہے۔
1,"""کی چینلوں پر براجمان """"ماہرین"""" قومی ٹیم کے پوسٹ مارٹم کا عمل بھی مکمل کر چکے ہیں ۔"""
2,ان کے لیے ہمدردی رکھتا ہوں جو مچھ سے زیادہ برے حال میں ہیں
3,لاہور دل کرتا ہے۔
4,گالیشیائی
5,سوئس ہائی جرمن
6,اروبا
7,اس طرح کی افواہیں پھیلانے
8,یقینا وہ ہوں۔
9,وہ جس کو چاہتا ملازمت پر رکھتا


In [35]:
dataset.push_to_hub(f"Cv_8_processed", split="train")

Let's download the data.

In [ ]:
# from datasets import load_dataset

# dataset = load_dataset("europarl_bilingual", lang1="en", lang2=target_lang, split="train")
# dataset

We see that the data is quite large - it has over a million translations. Since it's only text data, it should be relatively easy to process though.

Next, let's look at how the data was preprocessed when training the fine-tuned *XLS-R* checkpoint in Swedish. Looking at the [`run.sh` file](https://huggingface.co/hf-test/xls-r-300m-sv/blob/main/run.sh), we can see that the following characters were removed from the official transcriptions:

In [ ]:
# chars_to_ignore_regex = '[,?.!\-\;\:\"“%‘”�—’…–]'  # change to the ignored characters of your fine-tuned model

Let's do the same here so that the alphabet of our language model matches the one of the fine-tuned acoustic checkpoints.

We can write a single map function to extract the Swedish text and process it right away.

In [ ]:
# import re

# def extract_text(batch):
#     text = batch['sentence']
#     audio_array = batch['audio']['array']
#     audio_SR = batch['audio']['sampling_rate']
#     audio_path = batch['audio']['path']
#     path = batch['path']

#     batch['text'] = text
    
#     batch['audio']['array'] = audio_array
#     batch['audio']['sampling_rate'] = audio_SR
#     batch['audio']['path'] = audio_path

#     batch['path'] = path
#     return batch
#   text = batch["translation"][target_lang]
#   batch["text"] = re.sub(chars_to_ignore_regex, "", text.lower())
#   return batch

Let's apply the `.map()` function. This should take roughly 5 minutes.

In [ ]:
# dataset = dataset.map(extract_text, remove_columns=dataset.column_names)
# dataset

  0%|          | 0/7431 [00:00<?, ?ex/s]

Dataset({
    features: ['text'],
    num_rows: 7431
})

Great. Our dataset is already finished. Let's upload it to the Hub so that we can inspect and reuse it better.

You can log in by executing the following cell.

Next, we call 🤗 Hugging Face's [`push_to_hub`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=push#datasets.Dataset.push_to_hub) method to upload the dataset to the repo `"swedish_corpora_parliament_processed"`.

In [ ]:
dataset.push_to_hub(f"Cv_8_processed", split="train")

  0%|          | 0/8 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:2168: FutureWarning: `identical_ok` has no effect and is deprecated. It will be removed in 0.11.0.
  FutureWarning,


That was easy! The dataset viewer is automatically enabled when uploading a new dataset, which is very convenient. You can now directly inspect the dataset online. 

Feel free to look through our preprocessed dataset directly on [`hf-test/sv_corpora_parliament_processed`](https://huggingface.co/datasets/hf-test/sv_corpora_parliament_processed). Even if we are not a native speaker in Swedish, we can see that the data is well processed and seems clean.

Next, let's use the data to build a language model.

## **3. Build an *n-gram* with KenLM**

While large language models based on the [Transformer architecture](https://jalammar.github.io/illustrated-transformer/) have become the standard in NLP, it is still very common to use an ***n-gram*** LM to boost speech recognition systems - as shown in Section 1.

Looking again at Table 9 of Appendix C of the [official Wav2Vec2 paper](https://arxiv.org/abs/2006.11477), it can be noticed that using a *Transformer*-based LM for decoding clearly yields better results than using an *n-gram* model, but the difference between *n-gram* and *Transformer*-based LM is much less significant than the difference between *n-gram* and no LM. 

*E.g.*, for the large Wav2Vec2 checkpoint that was fine-tuned on 10min only, an *n-gram* reduces the word error rate (WER) compared to no LM by *ca.* 80% while a *Transformer*-based LM *only* reduces the WER by another 23% compared to the *n-gram*. This relative WER reduction becomes less, the more data the acoustic model has been trained on. *E.g.*, for the large checkpoint a *Transformer*-based LM reduces the WER by merely 8% compared to an *n-gram* LM whereas the *n-gram* still yields a 21% WER reduction compared to no language model.

The reason why an *n-gram* is preferred over a *Transformer*-based LM is that *n-grams* come at a significantly smaller computational cost. For an *n-gram*, retrieving the probability of a word given previous words is almost only as computationally expensive as querying a look-up table or tree-like data storage - *i.e.* it's very fast compared to modern *Transformer*-based language models that would require a full forward pass to retrieve the next word probabilities.

For more information on how *n-grams* function and why they are (still) so useful for speech recognition, the reader is advised to take a look at [this excellent summary](https://web.stanford.edu/~jurafsky/slp3/3.pdf) from Stanford.

Great, let's see step-by-step how to build an *n-gram*. We will use the popular [KenLM library](https://github.com/kpu/kenlm) to do so. Let's start by installing the Ubuntu library prerequisites:

In [ ]:
!sudo apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
libboost-program-options-dev is already the newest version (1.65.1.0ubuntu1).
libboost-program-options-dev set to manually installed.
libboost-system-dev is already the newest version (1.65.1.0ubuntu1).
libboost-system-dev set to manually installed.
libboost-thread-dev is already the newest version (1.65.1.0ubuntu1).
libboost-thread-dev set to manually installed.
libboost-test-dev is already the newest version (1.65.1.0ubuntu1).
libboost-test-dev set to manually installed.
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
libbz2-dev is already the newest version (1.0.6-8.1ubuntu0.2).
libbz2-dev set to manually installed.
liblzma-dev is already the newest version (5.2.2-1.3ubuntu0.1).
liblzma-dev set to manually installed.
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2.2).
zlib1g-dev set to manually in

before downloading and unpacking the KenLM repo.

In [ ]:
!wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz

--2022-09-11 19:53:28--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491888 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 480.36K   511KB/s    in 0.9s    

2022-09-11 19:53:29 (511 KB/s) - written to stdout [491888/491888]



KenLM is written in C++, so we'll make use of `cmake` to build the binaries.

In [ ]:
!mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
!ls kenlm/build/bin

Great, as we can see, the executable functions have successfully been built under `kenlm/build/bin/`.

KenLM by default computes an *n-gram* with [Kneser-Ney smooting](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing). All text data used to create the *n-gram* is expected to be stored in a text file.
We download our dataset and save it as a `.txt` file.

In [ ]:
from datasets import load_dataset

username = "BakhtUllah123"  # change to your username

dataset = load_dataset(f"{username}/Cv_8_processed", split="train")

with open("text.txt", "w") as file:
  file.write(" ".join(dataset["text"]))

Now, we just have to run KenLM's `lmplz` command to build our *n-gram*, called `"5gram.arpa"`. As it's relatively common in speech recognition, we build a *5-gram* by passing the `-o 5` parameter.
For more information on the different *n-gram* LM that can be built 
with KenLM, one can take a look at the [official website of KenLM](https://kheafield.com/code/kenlm/).

Executing the command below might take a minute or so.

In [ ]:

!kenlm/build/bin/lmplz -o 2 <"text.txt" > "2gram.arpa"

=== 1/5 Counting and sorting n-grams ===
Reading /content/text.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 1917870080 bytes == 0x557861b1c000 @  0x7f7d6f1301e7 0x5578602ea7e2 0x5578602854fe 0x5578602642eb 0x557860250066 0x7f7d6d2c9c87 0x557860251baa
tcmalloc: large alloc 8950054912 bytes == 0x5578d4022000 @  0x7f7d6f1301e7 0x5578602ea7e2 0x5578602d980a 0x5578602da248 0x557860264308 0x557860250066 0x7f7d6d2c9c87 0x557860251baa
****************************************************************************************************
Unigram tokens 61314 types 9307
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:111684 2:1062729408 3:1992617856 4:3188188416 5:4649441792
tcmalloc: large alloc 4649443328 bytes == 0x557861b1c000 @  0x7f7d6f1301e7 0x5578602ea7e2 0x5578602d980a 0x5578602da248 0x5578602648d7 0x557860250066 0x7f7d6d2c9c87 0x557860251baa
tcmalloc: large alloc 1992622080 bytes == 0x5579

Great, we have built a *5-gram* LM! Let's inspect the first couple of lines.

In [ ]:
!head -20 2gram.arpa

\data\
ngram 1=9306
ngram 2=41626
ngram 3=56650
ngram 4=59488
ngram 5=60134

\1-grams:
-4.6481605	<unk>	0
0	<s>	-0.07590369
-4.3479195	کروڑوں	-0.07590369
-3.4586525	بچے	-0.10982248
-2.796686	ہر	-0.21666135
-4.509916	روزسکول	-0.07590369
-2.859862	جا	-0.4558612
-4.225242	تے	-0.07590369
-2.3087177	ہیں۔	-0.23515454
-1.7079524	میں	-0.25884476
-2.7358274	اپنی	-0.11986762
-4.07802	خواہشات	-0.16625261


There is a small problem that 🤗 Transformers will not be happy about later on.
The *5-gram* correctly includes a "Unknown" or `<unk>`, as well as a *begin-of-sentence*, `<s>` token, but no *end-of-sentence*, `</s>` token.
This sadly has to be corrected currently after the build.

We can simply add the *end-of-sentence* token by adding the line `0 </s>  -0.11831701` below the *begin-of-sentence* token and increasing the `ngram 1` count by 1. Because the file has roughly 100 million lines, this command will take *ca.* 2 minutes.

In [ ]:
with open("2gram.arpa", "r") as read_file, open("2gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)

Let's now inspect the corrected *5-gram*.

In [ ]:
!head -20 2gram_correct.arpa

\data\
ngram 1=9307
ngram 2=41626
ngram 3=56650
ngram 4=59488
ngram 5=60134

\1-grams:
-4.6481605	<unk>	0
0	<s>	-0.07590369
0	</s>	-0.07590369
-4.3479195	کروڑوں	-0.07590369
-3.4586525	بچے	-0.10982248
-2.796686	ہر	-0.21666135
-4.509916	روزسکول	-0.07590369
-2.859862	جا	-0.4558612
-4.225242	تے	-0.07590369
-2.3087177	ہیں۔	-0.23515454
-1.7079524	میں	-0.25884476
-2.7358274	اپنی	-0.11986762


Great, this looks better! We're done at this point and all that is left to do is to correctly integrate the `"ngram"` with [`pyctcdecode`](https://github.com/kensho-technologies/pyctcdecode) and 🤗 Transformers.

## **4. Combine an *n-gram* with Wav2Vec2**

In a final step, we want to wrap the *5-gram* into a `Wav2Vec2ProcessorWithLM` object to make the *5-gram* boosted decoding as seamless as shown in Section 1.
We start by downloading the currently "LM-less" processor of [`xls-r-300m-sv`](https://huggingface.co/hf-test/xls-r-300m-sv).

In [ ]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("BakhtUllah123/xls-r_cv_ur")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:92: FutureWarning: Deprecated argument(s) used in 'snapshot_download': allow_regex. Will not be supported from version '0.12'.

Please use `allow_patterns` and `ignore_patterns` instead.
  warnings.warn(message, FutureWarning)


Downloading:   0%|          | 0.00/528 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/101k [00:00<?, ?B/s]

Next, we extract the vocabulary of its tokenizer as it represents the `"labels"` of `pyctcdecode`'s `BeamSearchDecoder` class.

In [ ]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

The `"labels"` and the previously built `5gram_correct.arpa` file is all that's needed to build the decoder. 

In [ ]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="2gram_correct.arpa",
)

We can safely ignore the warning and all that is left to do now is to wrap the just created `decoder`, together with the processor's `tokenizer` and `feature_extractor` into a `Wav2Vec2ProcessorWithLM` class.

In [ ]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

We want to directly upload the LM-boosted processor into 
the model folder of [`xls-r-300m-sv`](https://huggingface.co/hf-test/xls-r-300m-sv) to have all relevant files in one place.

Let's clone the repo, add the new decoder files and upload them afterward.
First, we need to install `git-lfs`.

In [ ]:
!sudo apt-get install git-lfs tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 1s (46.1 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: fall

Cloning and uploading of modeling files can be done conveniently with the `huggingface_hub`'s `Repository` class. 

More information on how to use the `huggingface_hub` to upload any files, please take a look at the [official docs](https://huggingface.co/docs/hub/how-to-upstream).

In [ ]:
from huggingface_hub import Repository

repo = Repository(local_dir="xls-r-53_ur", clone_from="BakhtUllah123/xls-r_cv_ur")

Cloning https://huggingface.co/omar47/wav2vec2-large-xls-r-300m-urdu-cv-10 into local empty directory.


Download file pytorch_model.bin:   0%|          | 1.51k/1.18G [00:00<?, ?B/s]

Download file runs/Aug07_11-52-33_13eca4db5f87/1659873280.2154987/events.out.tfevents.1659873280.13eca4db5f87.…

Download file runs/Aug07_11-52-33_13eca4db5f87/events.out.tfevents.1659873280.13eca4db5f87.71.0:   6%|5       …

Download file training_args.bin: 100%|##########| 3.30k/3.30k [00:00<?, ?B/s]

...

Clean file runs/Aug07_11-52-33_13eca4db5f87/1659873280.2154987/events.out.tfevents.1659873280.13eca4db5f87.71.…

Clean file training_args.bin:  30%|###       | 1.00k/3.30k [00:00<?, ?B/s]

Clean file runs/Aug07_11-52-33_13eca4db5f87/events.out.tfevents.1659873280.13eca4db5f87.71.0:   3%|3         |…

Clean file language_model/5gram.bin:   0%|          | 1.00k/4.97M [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/1.18G [00:00<?, ?B/s]

Having cloned `xls-r-300m-sv`, let's save the new processor with LM into it.

In [ ]:
processor_with_lm.save_pretrained("xls-r-53_ur")

Let's inspect the local repository. The `tree` command conveniently can also show the size of the different files.

In [ ]:
!tree -h xls-r-53_ur/

xls-r-300m-ur/
├── [  30]  added_tokens.json
├── [ 528]  alphabet.json
├── [2.0K]  config.json
├── [4.0K]  language_model
│   ├── [ 11M]  5gram_correct.arpa
│   ├── [  78]  attrs.json
│   └── [ 99K]  unigrams.txt
├── [ 262]  preprocessor_config.json
├── [1.2G]  pytorch_model.bin
├── [5.0K]  README.md
├── [4.0K]  runs
│   └── [4.0K]  Aug07_11-52-33_13eca4db5f87
│       ├── [4.0K]  1659873280.2154987
│       │   └── [5.3K]  events.out.tfevents.1659873280.13eca4db5f87.71.1
│       └── [ 33K]  events.out.tfevents.1659873280.13eca4db5f87.71.0
├── [ 952]  special_tokens_map.json
├── [ 414]  tokenizer_config.json
├── [3.3K]  training_args.bin
└── [ 597]  vocab.json

4 directories, 15 files


As can be seen the *5-gram* LM is quite large - it amounts to more than 4 GB.
To reduce the size of the *n-gram* and make loading faster, `kenLM` allows converting `.arpa` files to binary ones using the `build_binary` executable.

Let's make use of it here.

In [ ]:
!kenlm/build/bin/build_binary xls-r-53_ur/language_model/2gram_correct.arpa xls-r-53_ur/language_model/2gram.bin

Reading xls-r-300m-ur/language_model/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


Great, it worked! Let's remove the `.arpa` file and check the size of the binary *5-gram* LM.

In [ ]:
!rm xls-r-53_ur/language_model/2gram_correct.arpa && tree -h xls-r-53_ur/

xls-r-300m-ur/
├── [  30]  added_tokens.json
├── [ 528]  alphabet.json
├── [2.0K]  config.json
├── [4.0K]  language_model
│   ├── [5.0M]  5gram.bin
│   ├── [  78]  attrs.json
│   └── [ 99K]  unigrams.txt
├── [ 262]  preprocessor_config.json
├── [1.2G]  pytorch_model.bin
├── [5.0K]  README.md
├── [4.0K]  runs
│   └── [4.0K]  Aug07_11-52-33_13eca4db5f87
│       ├── [4.0K]  1659873280.2154987
│       │   └── [5.3K]  events.out.tfevents.1659873280.13eca4db5f87.71.1
│       └── [ 33K]  events.out.tfevents.1659873280.13eca4db5f87.71.0
├── [ 952]  special_tokens_map.json
├── [ 414]  tokenizer_config.json
├── [3.3K]  training_args.bin
└── [ 597]  vocab.json

4 directories, 15 files


Nice, we reduced the *n-gram* by more than half to less than 2GB now. In the final step, let's upload all files.

In [ ]:
repo.push_to_hub(commit_message="Upload lm- decoder")

That's it. Now you should be able to use the *5gram* for LM-boosted decoding as shown in Section 1.

As can be seen on [`xls-r-300m-sv`'s model card](https://huggingface.co/hf-test/xls-r-300m-sv#inference-with-lm) our *5gram* LM-boosted decoder yields a WER of 18.85% on Common Voice's 7 test set which is a relative performance of *ca.* 30% 🔥.